In [2]:
import pandas as pd
import numpy as np
from operator import itemgetter 
from scipy.stats import gamma

In [3]:
class BootStrapChian:
    def __init__(self, df_t, R):
        self.df_t = df_t
        self.R = R
        self.IBNR_all = pd.DataFrame()


    def Union(self,lst1, lst2):
        final_list = list(set(lst1) | set(lst2))
        return final_list
        
    def delete_element_list(self,list_1,list_2):
        for item in list_2:
            if item in list_1:
                list_1.remove(item)
        return(list_1)

    def index_all(self,df_t):
        df_trian = self.df_t
        index_he = []
        index_ost = []
        index_firs = []
        m = df_trian.shape[0]
        n = df_trian.shape[1]
        for i in range(n-1):
            ind_row = [x for x in range(m)]
            ind_col_last = np.where(df_trian.iloc[:,i+1].isnull())[0].tolist()
            ind_col_before = np.where(df_trian.iloc[:,i].isnull())[0].tolist()
            index_ost.append(self.delete_element_list(ind_row,ind_col_before).pop())
            index_firs.append(min(self.delete_element_list(ind_row,ind_col_last)))
            sum_ind = self.Union(ind_col_last,ind_col_before)
            dev_ind = self.delete_element_list(ind_row,sum_ind)
            index_he.append(dev_ind)
        index_ost.append(ind_col_last[0]-1)
        return(index_he,index_ost,index_firs)

    def wyznacz_f_1(self,df_triangle):
        indeksy, _ ,_= self.index_all(df_triangle)
        f = []
        n = df_triangle.shape[1]
        for i in range(n-1):
            f.append(sum(df_triangle.iloc[indeksy[i],i+1].dropna().astype(float))/sum(df_triangle.iloc[indeksy[i],i].dropna().astype(float)))
        return(f)    

    def trian_diag(self,df_triangle):
        n = df_triangle.shape[1]
        _, ind_ost,_ = self.index_all(df_triangle)
        el = [df_triangle.iloc[i,j] for i,j in zip(ind_ost,range(n))]
        return(el)

    def reverse_list(self,arr):
        left = 0
        right = len(arr)-1
        while (left < right):
            # Swap
            temp = arr[left]
            arr[left] = arr[right]
            arr[right] = temp
            left += 1
            right -= 1
        return(arr)
    
    def iloczn_wstepujacy(self,lista):
        lista_new = []
        lista_new.append(lista[len(lista)-1])
        for i in range(len(lista)-1):
            lista_new.append(lista_new[i]*lista[len(lista)-2-i])
        return(lista_new)
    
    def add_el_list(self,a,b):
        pp = []
        for i in range(a,b+1,1):
            pp.append(i)
        return(pp)
    
    def row_list_all(self,list1,list2,nn):
        krotka = list(zip(list1,list2))
        row_list = []
        for i in range(nn-1):
            kr = krotka[nn-i-2]
            row_list.append(self.add_el_list(kr[0],kr[1]))
        return(row_list)
    
    def triangle_back(self,df_t,l1,l2,ults,f_p):
        n = df_t.shape[1]
        df_t_copy = df_t.copy()
        row_list = self.row_list_all(l1,l2,n)
        for j in range(n-1):
            y = f_p[j]
            df_t_copy.iloc[row_list[j],n-j-2] = [x/y for x in list(itemgetter(*row_list[j])(ults))] 
        return(df_t_copy)
    
    def incremental_triangle(self,df_triangle):
        indeksy, _ ,_= self.index_all(df_triangle)
        n = df_triangle.shape[1]
        df_trian_copy = df_triangle.copy()
        for i in range(n-1):
            b = df_triangle.iloc[indeksy[i],i+1] - df_triangle.iloc[indeksy[i],i]
            df_trian_copy.iloc[indeksy[i],i+1] = b
        return(df_trian_copy)
    
    def triangle_back_down_triangle2(self,df_t,l2,ult,f_p):
        ult_cop = ult
        m = df_t.shape[0]-1
        n = df_t.shape[1]
        f_p_rev = (f_p)
        df_t_copy = df_t.copy()
        df_t_copy[:] = np.nan
        len_org = len(df_t.iloc[:,n-1])
        len_ost = len(ult)
        r = len_org-len_ost
        ult_copy = ult
        r_copy = r
        if r>0:
            for i in range(r):
                ult_copy.insert(i,0)
        df_t_copy.iloc[:,n-1] = ult_copy
        for i in range(n-r_copy):
            ind_row_i = [a for a in range(r,m+1,1)]
            y = f_p_rev[i]

           # print(ind_row_i)
            if len(ind_row_i) >1:
                l_pom = [x/y for x in list(itemgetter(*ind_row_i)(ult_copy))]
                for j in range(r):
                    l_pom.insert(j,0)


                df_t_copy.iloc[:,n-i-2] = l_pom
            else:
                l_pom = [ult_copy[ind_row_i[0]]/y]
                for i in range(r):
                    l_pom.insert(i,0)
                df_t_copy.iloc[:,n-i-2] = l_pom
            r = r+1
        return(df_t_copy)

    def rev_incremental(self,dd):
        dd_copy = dd.copy()
        n = dd_copy.shape[1]
        dd_0 = dd_copy.fillna(0)  
        dd_copy_0 = dd_copy.fillna(0)  
        for i in range(n-1):
            dd_copy_0.iloc[:,i+1] = dd_0.iloc[:,i+1] - dd_0.iloc[:,i]
        return(dd_copy_0)
    
    def elements_triangle(self,df_dd,ult2):
        out_list = []
        ul_cop2 = ult2
        m = df_dd.shape[0]-1
        n = df_dd.shape[1]
        len_org = len(df_dd.iloc[:,n-1])
        len_ost = len(ult2)
        r = len_org-len_ost
        ind_row_i = []
        r_copy = r
        for i in range(n-r_copy):
            ind_row_i.append([a for a in range(r,m+1,1)])
            r = r+1
        for i in range(n-r_copy):
            df_dd.iloc[:,i+r_copy]
            for x in df_dd.iloc[ind_row_i[n-r_copy-i-1],i+r_copy].tolist():
                out_list.append(x)
        return(out_list)
    
    def add_gamma_to_pdf(self,df_dd,ult2,w_gamma):
        znak_triangle = np.sign(df_dd)
        process_triangl = pd.DataFrame(0, columns=znak_triangle.columns, index=znak_triangle.index)
        out_list = []
        ul_cop2 = ult2
        m = df_dd.shape[0]-1
        n = df_dd.shape[1]
        len_org = len(df_dd.iloc[:,n-1])
        len_ost = len(ult2)
        r = len_org-len_ost
        r_copy = r
        k = 0
        for i in range(n-r_copy):
            ind_row_indy = [a for a in range(m-r+1,m+1,1)]
            r = r+1
            for j in ind_row_indy:
                if(znak_triangle.iloc[j,i+r_copy]==1.0):
                    process_triangl.iloc[j,i+r_copy] = w_gamma[k]
                elif(znak_triangle.iloc[j,i+r_copy]==-1.0):
                    process_triangl.iloc[j,i+r_copy] = -w_gamma[k]
                k = k+1
        return(process_triangl)

    def Boot_strap(self):
        m = self.df_t.shape[0]
        n = self.df_t.shape[1]
        f = self.wyznacz_f_1(self.df_t)
        f_prod = self.iloczn_wstepujacy(f)
        diagonala = self.trian_diag(self.df_t)
        diagonala = diagonala[0:(len(f_prod))]
        diagonala_rev = self.reverse_list(diagonala)
        n = self.df_t.shape[1]
        ults = [diagonala_rev[i]*f_prod[j] for i,j in zip(range(n-1),range(n-1))]
        len_org = len(self.df_t.iloc[:,n-1])
        len_est = len(ults)
        r = len_org-len_est
        if r>0:
            for i in range(r):
                ults.insert(i,self.df_t.iloc[i,n-1])
        _,ind_dol,ind_gora = self.index_all(self.df_t)
        l1,l2 = ind_gora,ind_dol[:-1]        
        back_tr = self.triangle_back(self.df_t,l1,l2,ults,f_prod)
        exp_inc_triangle = self.incremental_triangle(back_tr)
        inc_triangle = self.incremental_triangle(self.df_t) 
        nobs = 0.5*(self.df_t.shape[1])*(self.df_t.shape[1]+1)
        scale_factor = (nobs - 2*self.df_t.shape[1]+1)
        res_triangle = (inc_triangle - exp_inc_triangle)/np.sqrt(np.abs(exp_inc_triangle))
        res_triangle = round(res_triangle,5)
        adj_res = res_triangle*round(np.sqrt(nobs/scale_factor),6)
        scale_phi = (np.sum(res_triangle**2)/scale_factor).sum() 
        df = pd.DataFrame(
                    columns =list(self.df_t.index))
        df["Suma"] = []
        for iteracje in range(self.R):
                diagonala_rev = ''
                diagonala_p = ''
                choices = adj_res.values[~pd.isnull(adj_res.values)]
                losowa_macierz = adj_res.applymap(lambda x: np.random.choice(choices) if not pd.isnull(x) else x)
                new_triangle = losowa_macierz*np.sqrt(np.abs(exp_inc_triangle))+exp_inc_triangle
                suma_kum = new_triangle.cumsum(axis = 1)
                f_przejscia = self.wyznacz_f_1(suma_kum)
                f_prod = self.iloczn_wstepujacy(f_przejscia)
                diagonala = self.trian_diag(suma_kum)
                diagonala_p = diagonala[0:(len(f_prod))]
                diagonala_rev = self.reverse_list(diagonala_p)
                n = self.df_t.shape[1]
                ults_loop = [diagonala_rev[i]*f_prod[j] for i,j in zip(range(n-1),range(n-1))]
                tr_down = self.triangle_back_down_triangle2(self.df_t,l2,ults_loop,f_prod)
                ults_loop_2 = [diagonala_rev[i]*f_prod[j] for i,j in zip(range(n-1),range(n-1))]
                rev_triangle = self.rev_incremental(tr_down)
                list_element = self.elements_triangle(rev_triangle,ults_loop_2)
                scale = [np.abs(x/scale_phi) for x in list_element]
                wartosci_gamma = gamma.rvs(scale, scale=scale_phi, size=len(scale))
                processTriangle = self.add_gamma_to_pdf(rev_triangle,ults_loop_2,wartosci_gamma)
                IBNR = processTriangle.cumsum(axis = 1)
                df.loc[len(df.index)] = IBNR.iloc[:,n-1].to_list() + [np.sum(IBNR.iloc[:,n-1].to_list())]
        IBNR_all = df.copy()
        return(IBNR_all)